In [54]:
%load_ext autoreload
%autoreload 2

# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import re
import sys
import time
import math
import numpy as np
from datetime import datetime
import tensorflow as tf

sys.path.append(os.path.join(os.getcwd(), '..', 'utils'))
from utils import *
from tf_utils import *
import cifar10_loader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This TensorFlow tutorial is written based on https://github.com/tensorflow/models/tree/master/tutorials/image/cifar10, which is the tutorial for the image classification task on CIFAR-10 dataset.

In [2]:
# Basic model parameters.
batch_size = 128
data_dir = 'cifar10_data'
data_url = 'http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'

# Global constants describing the CIFAR-10 data set.
IMAGE_SIZE = cifar10_loader.IMAGE_SIZE
NUM_CLASSES = cifar10_loader.NUM_CLASSES
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = cifar10_loader.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = cifar10_loader.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

# If a model is trained with multiple GPUs, prefix all Op names with tower_name
# to differentiate the operations. Note that this prefix is removed from the
# names of the summaries when visualizing a model.
TOWER_NAME = 'tower'

### Download the CIFAR-10 dataset

In [20]:
maybe_download_and_extract(data_url, data_dir, 'cifar-10-batches-bin')

>> Downloading cifar-10-binary.tar.gz 100.0%('Successfully downloaded', 'cifar-10-binary.tar.gz', 170052171, 'bytes.')


### Get variable initialization function

In [3]:
initializer = get_initializer('normal', **{'stddev':0.01})

### Functions for constructing model (graph)

In [7]:
def build_inputs(data_dir, eval_data=False):
    """Construct input for CIFAR evaluation using the Reader ops.
    Args:
        data_dir: path to directory including data
        eval_data: bool, indicating if one should use the train or eval data set.
    Returns:
        images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
        labels: Labels. 1D tensor of [batch_size] size.
    Raises:
        ValueError: If no data_dir
    """
    if not data_dir:
        raise ValueError('Please supply a data_dir')
    data_dir = os.path.join(data_dir, 'cifar-10-batches-bin')
    images, labels = cifar10_loader.inputs(eval_data=eval_data,
                                        data_dir=data_dir,
                                        batch_size=batch_size)
    
    #images, labels = cifar10_loader.distorted_inputs(data_dir=data_dir,
    #                                              batch_size=batch_size)
    
    return images, labels


def build_model(images):
    """Build the CIFAR-10 model.
    Args:
        images: Images returned from distorted_inputs() or inputs().
    Returns:
        Logits.
    """
    # We instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs.
    # If we only ran this model on a single GPU, we could simplify this function
    # by replacing all instances of tf.get_variable() with tf.Variable().
    
    # conv1
    with tf.variable_scope('conv1') as scope:
        conv1 = get_conv2D_layer(images, 3, 64, 5, 1, initializer, 0.0, 'relu', scope, True)

    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        conv2 = get_conv2D_layer(norm1, 64, 64, 5, 1, initializer, 0.0, 'relu', scope, True)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # fc1
    with tf.variable_scope('fc1') as scope:
        fc1 = get_fully_connected_layer(pool2, -1, 384, initializer, 0.004, 'relu', 
                                        True, batch_size, scope, True)

    # fc2
    with tf.variable_scope('fc2') as scope:
        fc2 = get_fully_connected_layer(fc1, 384, 192, initializer, 0.004, 'relu', 
                                        False, -1, scope, True)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        softmax_linear = get_fully_connected_layer(fc2, 192, NUM_CLASSES, initializer, 
                                                   0.0, 'None', False, -1, scope, True)

    return softmax_linear


def build_loss(logits, labels):
    """Add L2Loss to all the trainable variables.
    Add summary for "Loss" and "Loss/avg".
    Args:
        logits: Logits from inference().
        labels: Labels from distorted_inputs or inputs(). 1-D tensor
            of shape [batch_size]
    Returns:
        Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def _add_loss_summaries(total_loss):
    """Add summaries for losses in CIFAR-10 model.
    Generates moving average for all losses and associated summaries for
    visualizing the performance of the network.
    Args:
        total_loss: Total loss from loss().
    Returns:
        loss_averages_op: op for generating moving averages of losses.
    """
    # Compute the moving average of all individual losses and the total loss.
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
        # Name each loss as '(raw)' and name the moving average version of the loss
        # as the original loss name.
        tf.summary.scalar(l.op.name + ' (raw)', l)
        tf.summary.scalar(l.op.name, loss_averages.average(l))

    return loss_averages_op


def build_train_operation(total_loss, global_step):
    """Train CIFAR-10 model.
    Create an optimizer and apply to all trainable variables. Add moving
    average for all trainable variables.
    Args:
        total_loss: Total loss from loss().
        global_step: Integer Variable counting the number of training steps
            processed.
    Returns:
        train_op: op for training.
    """
    # Variables that affect learning rate.
    num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / batch_size
    decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)

    # Decay the learning rate exponentially based on the number of steps.
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                  global_step,
                                  decay_steps,
                                  LEARNING_RATE_DECAY_FACTOR,
                                  staircase=True)
    tf.summary.scalar('learning_rate', lr)

    # Generate moving averages of all losses and associated summaries.
    loss_averages_op = _add_loss_summaries(total_loss)

    # Compute gradients.
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)

    # Apply gradients.
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')

    return train_op

### Training the model

In [ ]:
train_dir = 'cifar10_train'
max_steps = 10000
log_device_placement = False
log_frequency = 10

if tf.gfile.Exists(train_dir):
    tf.gfile.DeleteRecursively(train_dir)
else:
    tf.gfile.MakeDirs(train_dir)

In [21]:
"""Train CIFAR-10 for a number of steps."""
with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    # Force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    with tf.device('/cpu:0'):
        images, labels = build_inputs(data_dir, False)

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = build_model(images)

    # Calculate loss.
    loss = build_loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = build_train_operation(loss, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
        """Logs loss and runtime."""

        def begin(self):
            self._step = -1
            self._start_time = time.time()

        def before_run(self, run_context):
            self._step += 1
            return tf.train.SessionRunArgs(loss)  # Asks for loss value.

        def after_run(self, run_context, run_values):
            if self._step % log_frequency == 0:
                current_time = time.time()
                duration = current_time - self._start_time
                self._start_time = current_time

                loss_value = run_values.results
                examples_per_sec = log_frequency * batch_size / duration
                sec_per_batch = float(duration / log_frequency)

                format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                    'sec/batch)')
                print (format_str % (datetime.now(), self._step, loss_value,
                    examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
            hooks=[tf.train.StopAtStepHook(last_step=max_steps),
            tf.train.NanTensorHook(loss), _LoggerHook()],
            config=tf.ConfigProto(log_device_placement=log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)


INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.


INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.


INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using conv2/weight_loss__raw_ instead.


INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using conv2/weight_loss__raw_ instead.


INFO:tensorflow:Summary name fc1/weight_loss (raw) is illegal; using fc1/weight_loss__raw_ instead.


...

INFO:tensorflow:global_step/sec: 30.3464


2017-06-16 02:21:29.242728: step 9800, loss = 0.44 (3884.3 examples/sec; 0.033 sec/batch)
INFO:tensorflow:global_step/sec: 30.2834


INFO:tensorflow:global_step/sec: 30.2834


2017-06-16 02:21:32.545947: step 9900, loss = 0.48 (3875.0 examples/sec; 0.033 sec/batch)
INFO:tensorflow:Saving checkpoints for 10000 into cifar10_train/model.ckpt.


INFO:tensorflow:Saving checkpoints for 10000 into cifar10_train/model.ckpt.


### Evaluating the model

In [49]:
eval_dir = 'cifar10_eval'
eval_data = 'test'
checkpoint_dir = 'cifar10_train'
eval_interval_secs = 1
num_examples = 10000
run_once = False

In [50]:
def eval_once(saver, summary_writer, top_k_op, summary_op):
    """Run Eval once.
    Args:
    saver: Saver.
    summary_writer: Summary writer.
    top_k_op: Top K op.
    summary_op: Summary op.
    """
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        print(ckpt)
        if ckpt and ckpt.model_checkpoint_path:
            # Restores from checkpoint
            saver.restore(sess, ckpt.model_checkpoint_path)
            # Assuming model_checkpoint_path looks something like:
            #   /my-favorite-path/cifar10_train/model.ckpt-0,
            # extract global_step from it.
            global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        else:
            print('No checkpoint file found')
            return

        # Start the queue runners.
        coord = tf.train.Coordinator()
        try:
            threads = []
            for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
                threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                                 start=True))

            num_iter = int(math.ceil(num_examples / batch_size))
            true_count = 0  # Counts the number of correct predictions.
            total_sample_count = num_iter * batch_size
            step = 0
            while step < num_iter and not coord.should_stop():
                predictions = sess.run([top_k_op])
                true_count += np.sum(predictions)
                step += 1

            # Compute precision @ 1.
            precision = true_count / total_sample_count
            print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

            summary = tf.Summary()
            summary.ParseFromString(sess.run(summary_op))
            summary.value.add(tag='Precision @ 1', simple_value=precision)
            summary_writer.add_summary(summary, global_step)
        except Exception as e:  # pylint: disable=broad-except
            coord.request_stop(e)

        coord.request_stop()
        coord.join(threads, stop_grace_period_secs=10)

In [55]:
with tf.Graph().as_default() as g:
    eval_data = eval_data == 'test'
    images, labels = build_inputs(data_dir, eval_data)
    
    logits = build_model(images)
    top_k_op = tf.nn.in_top_k(logits, labels, 1)
    
    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(eval_dir, g)

    while True:
        eval_once(saver, summary_writer, top_k_op, summary_op)
        if run_once:
            break
        time.sleep(eval_interval_secs)

model_checkpoint_path: "cifar10_train/model.ckpt-10000"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-1"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-10000"

2017-06-16 03:01:18.671515: precision @ 1 = 0.996
model_checkpoint_path: "cifar10_train/model.ckpt-10000"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-1"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-10000"

2017-06-16 03:01:24.496865: precision @ 1 = 0.997
model_checkpoint_path: "cifar10_train/model.ckpt-10000"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-1"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-10000"

2017-06-16 03:01:30.323077: precision @ 1 = 0.997
model_checkpoint_path: "cifar10_train/model.ckpt-10000"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-1"
all_model_checkpoint_paths: "cifar10_train/model.ckpt-10000"

2017-06-16 03:01:36.167443: precision @ 1 = 0.995
model_checkpoint_path: "cifar10_train/model.ckpt-10000"
all_model_checkpoint_paths: "cifar10_train/m

INFO:tensorflow:Error reported to Coordinator: <type 'exceptions.RuntimeError'>, Attempted to use a closed Session.


KeyboardInterrupt: 